<a href="https://colab.research.google.com/github/MarcelodeFreitas/udemy_deep_learning_pytorch_python/blob/main/Projeto_3_Classifica%C3%A7%C3%A3o_bin%C3%A1ria_breast_cancer_com_tuning_de_par%C3%A2metros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 3: Classificação binária brest cancer com tuning dos parâmetros

## Etapa 1: Importação das bibliotecas

In [ ]:
!pip install skorch

     |████████████████████████████████| 155 kB 26.8 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import sklearn
import skorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetBinaryClassifier

In [ ]:
torch.__version__, skorch.__version__, sklearn.__version__

('1.10.0+cu111', '0.11.0', '1.0.2')

## Etapa 2: Base de dados

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')

In [ ]:
previsores = np.array(previsores, dtype = 'float32')
classe = np.array(classe, dtype = 'float32').squeeze(1)

In [ ]:
previsores.shape

(569, 30)

In [ ]:
classe.shape

(569,)

## Etapa 3: Classe para estrutura da rede neural

**\*\* ATUALIZAÇÃO JAN/2022 \*\*** : na versão atual do Skorch, os resultados da rede neural devem ser retornados sem ativação, ou seja, sem a camada sigmoide no final. Com isto, a função de custo deve ser `BCEWithLogitsLoss`.

In [ ]:
class classificador_torch(nn.Module):
  def __init__(self, activation, neurons, initializer):
    super().__init__()
    # 30 -> 16 -> 16 -> 1
    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation
    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation
    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)
    # self.output = nn.Sigmoid() ** ATUALIZAÇÃO (ver detalhes no texto acima) **

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dense2(X)
    # X = self.output(X) ** ATUALIZAÇÃO (ver detalhes no texto acima) **
    return X

## Etapa 4: Skorch

In [ ]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador_torch,
                                                  lr = 0.001,
                                                  optimizer__weight_decay = 0.0001,
                                                  train_split=False)

## Etapa 5: Tuning dos parâmetros

In [ ]:
params = {'batch_size': [10],
          'max_epochs': [100],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'criterion': [torch.nn.BCEWithLogitsLoss], #, torch.nn.HingeEmbeddingLoss], # ** ATUALIZAÇÃO **
          'module__activation': [F.relu, F.tanh],
          'module__neurons': [8, 16],
          'module__initializer': [torch.nn.init.uniform]} # _, torch.nn.init.normal_]}

In [ ]:
params

{'batch_size': [10],
 'criterion': [torch.nn.modules.loss.BCEWithLogitsLoss],
 'max_epochs': [100],
 'module__activation': [<function torch.nn.functional.relu>,
  <function torch.nn.functional.tanh>],
 'module__initializer': [<function torch.nn.init._make_deprecate.<locals>.deprecated_init>],
 'module__neurons': [8, 16],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD]}

In [ ]:
grid_search = GridSearchCV(estimator=classificador_sklearn, param_grid=params,
                           scoring = 'accuracy', cv = 2)
grid_search = grid_search.fit(previsores, classe)

  epoch    train_loss     dur
-------  ------------  ------
      1    15578.2424  0.0398
      2    13421.6141  0.0312
      3    11429.6435  0.0320
      4     9608.2269  0.0323
      5     7949.9496  0.0315


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if sys.path[0] == '':


      6     6437.1937  0.0348
      7     5047.1312  0.0339
      8     3753.3071  0.0334
      9     2525.4143  0.0330
     10     1327.9830  0.0355
     11      341.5486  0.0379
     12      120.7594  0.0315
     13      122.1383  0.0324
     14      128.3295  0.0324
     15      114.8607  0.0332
     16      118.6486  0.0321
     17      112.3182  0.0359
     18      109.7156  0.0342
     19      109.0899  0.0359
     20      103.8937  0.0320
     21      100.8837  0.0315
     22       98.9591  0.0340
     23       92.0899  0.0352
     24       93.9443  0.0327
     25       92.5766  0.0318
     26       90.4373  0.0316
     27       89.9594  0.0455
     28       86.8915  0.0380
     29       84.2178  0.0335
     30       85.4057  0.0321
     31       82.6213  0.0307
     32       79.1889  0.0318
     33       79.3554  0.0326
     34       77.9777  0.0314
     35       76.4595  0.0346
     36       74.0168  0.0323
     37       73.4926  0.0331
     38       65.6922  0.0336
     39   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if sys.path[0] == '':


      6    11952.6263  0.0342
      7    10466.9768  0.0319
      8     9137.4191  0.0314
      9     7942.1120  0.0317
     10     6860.1495  0.0319
     11     5871.7593  0.0359
     12     4959.5423  0.0321
     13     4107.4870  0.0331
     14     3302.5669  0.0296
     15     2560.3446  0.0309
     16     1868.8560  0.0305
     17     1179.0318  0.0303
     18      485.8732  0.0315
     19       62.1792  0.0319
     20       69.4658  0.0328
     21       61.8521  0.0331
     22       60.7808  0.0388
     23       58.8829  0.0353
     24       56.0241  0.0328
     25       54.3968  0.0327
     26       54.6734  0.0308
     27       53.7153  0.0323
     28       52.1105  0.0317
     29       49.6147  0.0333
     30       48.3194  0.0378
     31       46.2406  0.0326
     32       46.2065  0.0393
     33       47.3731  0.0315
     34       44.6859  0.0315
     35       44.8199  0.0321
     36       43.8066  0.0308
     37       43.2638  0.0311
     38       42.9636  0.0352
     39   

...

      7        2.1831  0.0347
      8        2.1142  0.0273
      9        2.0455  0.0265
     10        1.9771  0.0253
     11        1.9091  0.0280
     12        1.8415  0.0259
     13        1.7743  0.0298
     14        1.7078  0.0279
     15        1.6419  0.0256
     16        1.5769  0.0293
     17        1.5128  0.0267
     18        1.4499  0.0252
     19        1.3883  0.0259
     20        1.3282  0.0276
     21        1.2698  0.0241
     22        1.2135  0.0268
     23        1.1593  0.0258
     24        1.1076  0.0251
     25        1.0587  0.0285
     26        1.0127  0.0282
     27        0.9698  0.0260
     28        0.9302  0.0274
     29        0.8940  0.0261
     30        0.8612  0.0250
     31        0.8318  0.0292
     32        0.8058  0.0275
     33        0.7829  0.0273
     34        0.7630  0.0262
     35        0.7458  0.0293
     36        0.7312  0.0261
     37        0.7188  0.0325
     38        0.7084  0.0284
     39        0.6997  0.0283
     40   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


      7        1.5690  0.0288
      8        1.5079  0.0272
      9        1.4479  0.0265
     10        1.3891  0.0267
     11        1.3318  0.0274
     12        1.2760  0.0294
     13        1.2220  0.0275
     14        1.1701  0.0277
     15        1.1204  0.0325
     16        1.0731  0.0266
     17        1.0285  0.0264
     18        0.9867  0.0243
     19        0.9479  0.0266
     20        0.9122  0.0278
     21        0.8796  0.0314
     22        0.8500  0.0270
     23        0.8236  0.0271
     24        0.8001  0.0306
     25        0.7794  0.0267
     26        0.7613  0.0298
     27        0.7457  0.0257
     28        0.7322  0.0257
     29        0.7207  0.0254
     30        0.7109  0.0275
     31        0.7027  0.0300
     32        0.6958  0.0241
     33        0.6899  0.0263
     34        0.6851  0.0242
     35        0.6810  0.0270
     36        0.6777  0.0311
     37        0.6749  0.0257
     38        0.6726  0.0276
     39        0.6707  0.0269
     40   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if sys.path[0] == '':


      4    12189.1588  0.0648
      5     9284.4983  0.0650
      6     7005.7584  0.0683
      7     5228.4374  0.0631
      8     3817.6086  0.0642
      9     2638.6100  0.0662
     10     1582.4207  0.0687
     11      573.3917  0.0642
     12      142.2150  0.0617
     13      130.0759  0.0617
     14      119.0500  0.0677
     15      108.1460  0.0693
     16       97.9789  0.0617
     17       89.6681  0.0606
     18       82.0786  0.0661
     19       74.6605  0.0633
     20       69.1234  0.0615
     21       63.8514  0.0617
     22       58.5320  0.0623
     23       53.6302  0.0658
     24       48.6940  0.0698
     25       44.1509  0.0621
     26       38.9907  0.0610
     27       34.6079  0.0659
     28       30.9640  0.0615
     29       27.1113  0.0615
     30       24.6490  0.0609
     31       21.8499  0.0608
     32       19.5577  0.0662
     33       16.9065  0.0667
     34       16.8499  0.0623
     35       15.1129  0.0627
     36       14.7831  0.0605
     37   

In [ ]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [ ]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCEWithLogitsLoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu>,
 'module__initializer': <function torch.nn.init._make_deprecate.<locals>.deprecated_init>,
 'module__neurons': 8,
 'optimizer': torch.optim.adam.Adam}

In [ ]:
melhor_precisao

0.7540770941438102